In [1]:
import pandas as pd
import os
import matplotlib
import seaborn as sns
import numpy as np

import os
import sys
nb_dir = os.path.split(os.getcwd())[0]
if nb_dir not in sys.path:
    sys.path.append(nb_dir)
    
from src.data.preprocessing import *
from src.data.load_data_yoochoose import *

## Data preprocessing

In [2]:
events = load_events()

In [3]:
data_x,data_y = preprocessing(events,"visitorid","itemid")

In [2]:
data_x,data_y = load_processed_sparse()

In [3]:
train_x,test_x,train_y,test_y = data_split(data_x,data_y)

In [9]:
data_x.shape

(3702, 100)

In [7]:
data_y.shape

(3702, 7234)

In [8]:
train_x.shape

(2961, 100)

In [6]:
train_y.shape

(2961, 7234)

### Use limited amount of data for first experiments and debugging

#### train/test 2000/500

In [5]:
train_x = train_x[:2000]

In [6]:
train_y = np.asarray(train_y[:2000])

In [7]:
test_x = test_x[:500]

In [8]:
test_y = np.asarray(test_y[:500])

In [9]:
train_x.shape

(2000, 100)

In [10]:
test_x.shape

(500, 100)

In [11]:
train_y.shape

(2000, 84594)

In [12]:
test_y.shape

(500, 84594)

#### Models training

In [10]:
import keras
from src.models import model

Using TensorFlow backend.


In [11]:
mdl = model.LSTMrecommender(
    emb_input_dim = 1000,
    emb_output_dim = 200,
    lstm_units = 64,
    dense_output_dim = train_y.shape[1],
    dense_activation = 'relu'
)

In [12]:
mdl.compile(optimizer='adam',
    loss='binary_crossentropy',
    metrics=[
        keras.metrics.Precision(top_k=5)
    ]
)

In [14]:
test_x[:32]

array([[2.14581815e+08, 2.14550383e+08, 2.14699044e+08, ...,
        0.00000000e+00, 0.00000000e+00, 0.00000000e+00],
       [2.14685754e+08, 2.14685422e+08, 2.14587290e+08, ...,
        0.00000000e+00, 0.00000000e+00, 0.00000000e+00],
       [2.14668596e+08, 2.14537967e+08, 2.14509073e+08, ...,
        0.00000000e+00, 0.00000000e+00, 0.00000000e+00],
       ...,
       [2.14834865e+08, 2.14826925e+08, 2.14651427e+08, ...,
        0.00000000e+00, 0.00000000e+00, 0.00000000e+00],
       [2.14640407e+08, 2.14716982e+08, 2.14839774e+08, ...,
        0.00000000e+00, 0.00000000e+00, 0.00000000e+00],
       [2.14821341e+08, 2.14551617e+08, 2.14821313e+08, ...,
        0.00000000e+00, 0.00000000e+00, 0.00000000e+00]])

In [13]:
test = mdl.predict(test_x[:32])
test.shape

InvalidArgumentError:  indices[7,0] = 214826704 is not in [0, 1000)
	 [[node embedding_1/embedding_lookup (defined at C:\Users\koprl\Anaconda3\lib\site-packages\tensorflow_core\python\framework\ops.py:1751) ]] [Op:__inference_keras_scratch_graph_509]

Function call stack:
keras_scratch_graph


In [ ]:
callbacks = [
    keras.callbacks.TensorBoard(
        log_dir=os.path.join('..\logs', 'v2_binary_cross'),
        histogram_freq=1,
        profile_batch=0
    )
]

mdl.fit(
    x=train_x,
    y=train_y,
    batch_size=256,
    epochs=100,
    validation_data=(test_x, test_y),
    callbacks = callbacks
)

#### train/test 100/50

In [ ]:
train_x = train_x[:100]

In [ ]:
train_y = np.asarray(train_y[:100])

In [ ]:
test_x = test_x[:50]

In [ ]:
test_y = np.asarray(test_y[:50])

In [ ]:
train_x.shape

In [ ]:
test_x.shape

In [ ]:
train_y.shape

In [ ]:
test_y.shape

#### Model training

In [ ]:
mdl = model.LSTMrecommender(
    emb_input_dim = 1000,
    emb_output_dim = 100,
    lstm_units = 10,
    dense_output_dim = train_y.shape[1],
    dense_activation = 'relu'
)

In [ ]:
mdl.compile(optimizer=keras.optimizers.SGD(learning_rate=0.1),
    loss='binary_crossentropy',
    metrics=[
        keras.metrics.Precision(top_k=100)
    ]
)

In [ ]:
test = mdl.predict(test_x[:32])
test.shape

In [ ]:
callbacks = [
    keras.callbacks.TensorBoard(
        log_dir=os.path.join('..\logs', 'v3_binary_cross'),
        histogram_freq=1,
        profile_batch=0
    )
]

mdl.fit(
    x=train_x,
    y=train_y,
    batch_size=32,
    epochs=20,
    validation_data=(test_x, test_y),
    callbacks = callbacks
)

As it seems, model is able to learn. However, it is far away from optimal performance.